In [1]:
import sys
sys.path.append('../StableMotifs/')

import stablemotifs
import biolqm
import ginsim
import StableMotifs as sm
import PyBoolNet
import boolsim
import timeit
import timeout_decorator
from timeout_decorator import TimeoutError
import pandas as pd
import glob
from colomoto_jupyter import tabulate
import cabean
import pypint


Bad key "text.kerning_factor" on line 4 in
/home/david/anaconda2/envs/python_3_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


boolsim==0.5
cabean==1.0
colomoto-jupyter==0.7.1
ginsim==0.4.2
networkx==2.4
numpy==1.18.5
PyBoolNet==2.2.9
stablemotifs==1.0
sympy==1.5.1
timeout-decorator==0.4.1

In [2]:
PyBoolNet.version()

'2.2.9'

## Parameters of the bechmark

In [3]:

#the folder in which all models we whish to run are saved in the booleanned format with .booleannet extention. 
models_path='models_for_benchmark/'

#run_ens_size is the number of times each method is repeatedly run IN THE SAME ITERATION. 
#this means that the timeout has to be increased proporionally to account for all runs
#In the final output the minimum (best) runtime is saved out of all in the ensemble 
run_ens_size=1 

#timeout limit in seconds for each method (all runs within the run_ens_size range). 
#If the timeout limit is reached the output will be -1
timeout_limit=43200


#name of methods to be exluded from the run. This can be set manually if we know 
#that some methods are not relevant or take too much time
exclude_from_run=set([])#'sm_jgtz','stable_motifs_new','CABEAN', 'Pint','boolsim','bioLQM','PyBoolNet_Asp'])
#exclude_from_run=set(['bioLQM','PyBoolNet_Asp','PyBoolNet','Pint']) 
#uncomment the line abovr if you want only methods that look for complex attractors

#the excel file in which the dataframe summarizing the results will be saved.
#if it already exists new results will be added to it
output_file_name='timing_results.xlsx'


In [4]:
@timeout_decorator.timeout(timeout_limit)
def run_timed_test(method_runstring, setup, repeat):

    return timeit.repeat(method_runstring,setup=setup, repeat=repeat, number=1)

try: #if the file already exists we dont overwrite it
    result_df=pd.read_excel(output_file_name)
    result_df=result_df.reset_index(drop=True)
except FileNotFoundError:
    result_df=pd.DataFrame()
    
#grabbing all the .booleannet models from the folder
model_set=glob.glob(models_path+'*.booleannet')
#model_set=['PSO.txt', 'phase_switch.txt','myeloid.txt'] 

for model_file_name in model_set:
    print('\nRunning model %s'%model_file_name)
    
    base_setup_string='''import stablemotifs
import biolqm
import ginsim
import StableMotifs as sm
import PyBoolNet
import boolsim
import cabean
import pypint
import json
from colomoto_jupyter import tabulate


model_file_name='%s'
with open(model_file_name) as f:
    rules=f.read().replace('#BOOLEAN RULES','').strip()
biolqm_model = biolqm.load(model_file_name, "booleannet")
pypint_model=biolqm.to_pint(biolqm_model)'''%model_file_name

    method_runstrings={'sm_jgtz': "a=stablemotifs.load(biolqm_model, quiet=True).attractors\ntabulate(a).to_excel('attr_output/attr_output_sm_jgtz_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'stable_motifs_new': "ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)\nex.attractor_dataframe(ar).to_excel('attr_output/attr_output_stable_motifs_new_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'PyBoolNet': 'd=PyBoolNet.Attractors.compute_json(primes, "asynchronous")\nwith open(\'attr_output/attr_output_PyBoolNet_\'+model_file_name.split(\'.\')[0].split(\'/\')[-1]+\'.json\',\'w\') as fp:\n\tjson.dump(d,fp)',
                   
                   
 'boolsim': "a=boolsim.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_boolsim_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'bioLQM': "a=biolqm.fixpoints(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_bioLQM_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'PyBoolNet_Asp': "a=PyBoolNet.AspSolver.steady_states(primes)\ntabulate(a).to_excel('attr_output/attr_output_PyBoolNet_Asp_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'CABEAN': "a=cabean.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_CABEAN_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'Pint': "a=pypint.fixpoints(pypint_model)\ntabulate(a).to_excel('attr_output/attr_output_Pint_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')"}

    method_setups={'sm_jgtz':base_setup_string,

    'stable_motifs_new':base_setup_string+'''
max_simulate_size=20
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
import StableMotifs.Export as ex
''',

    'PyBoolNet':base_setup_string+'''
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
''',

    'boolsim':base_setup_string,

    'bioLQM':base_setup_string,

    'PyBoolNet_Asp':base_setup_string+'''
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
''',

    'CABEAN':base_setup_string,

    'Pint':base_setup_string}

    method_runtimes={i:[] for i in method_runstrings}
    
    
    for method_name in set(method_runstrings)-exclude_from_run:
        print('method:',method_name)
        try:
            method_runtimes[method_name]=run_timed_test(method_runstrings[method_name], setup=method_setups[method_name], repeat=run_ens_size)
        except Exception as e:
            print(str(e))
            method_runtimes[method_name].append(-1)
    runtime_minimums={k:min(method_runtimes[k]) for k in method_runtimes.keys()-exclude_from_run} 
    runtime_minimums['model']=model_file_name.split('.')[0].split('/')[-1]
    result_df=result_df.append([runtime_minimums],ignore_index=True)
    result_df=result_df.reset_index(drop=True)

    result_df.to_excel('timing_results.xlsx')


Running model models_for_benchmark/phase_switch.booleannet
method: bioLQM
method: PyBoolNet_Asp
method: PyBoolNet
Attractors.compute_json(..)
 Attractors.completeness(..) yes
 working on minimal trapspace 1/3: 01100110110
  Attractors.univocality(..) yes
  Attractors.faithfulness(..) yes
  Attractors.find_attractor_state_by_randomwalk_and_ctl(..)
 working on minimal trapspace 2/3: 00010000010
  Attractors.univocality(..) yes
  Attractors.faithfulness(..) yes
  Attractors.find_attractor_state_by_randomwalk_and_ctl(..)
 working on minimal trapspace 3/3: 00101011101
  Attractors.univocality(..) yes
  Attractors.faithfulness(..) yes
  Attractors.find_attractor_state_by_randomwalk_and_ctl(..)
method: boolsim
method: sm_jgtz
method: Pint
method: CABEAN
method: stable_motifs_new


In [5]:
result_df

,bioLQM,PyBoolNet_Asp,PyBoolNet,boolsim,sm_jgtz,Pint,CABEAN,stable_motifs_new,model
0,0.304213,0.113231,0.392621,0.166109,31.469688,0.098801,0.133288,0.602926,phase_switch


result_df.to_excel(output_file_name)